#Setup

In [1]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

##exclude_ids.txt

In [ ]:
149633CM
212539MU
231555ZZ
704275DC

##Side Inputs

Co to jest side input?

Jak sama nazwa wskazuje, jest to dodatkowa informacja, którą możemy przekazać do obiektu DoFn.

Oprócz głównej PCollection jako wejścia, możemy wstrzyknąć dodatkowe dane do ParDo lub jego pochodnych transformacji, takich jak Map i FlatMap, w formie side inputs.

ParDo traktuje side input jako dodatkowe wejście, do którego ma dostęp za każdym razem, gdy przetwarza element w PCollection.

In [9]:
import apache_beam as beam  # Importujemy Apache Beam do budowy potoku przetwarzania danych

# Tworzymy pustą listę, w której przechowamy identyfikatory pracowników do wykluczenia
side_list = list()

# Otwieramy plik 'exclude_ids.txt', który zawiera listę ID do wykluczenia
with open('exclude_ids.txt', 'r') as my_file:
    for line in my_file:
        # Usuwamy białe znaki (np. '\n' z końca każdej linii) i dodajemy ID do listy
        side_list.append(line.rstrip())

# Tworzymy obiekt potoku Beam
p = beam.Pipeline()

# Definiujemy klasę DoFn, która będzie używana w transformacji ParDo do filtrowania danych
class FilterUsingLength(beam.DoFn):
    def process(self, element, side_list, lower_bound, upper_bound=float('inf')):
        """
        Metoda process() wykonuje operację filtrowania na każdym elemencie PCollection.
        - element: pojedynczy wiersz wejściowych danych jako string.
        - side_list: lista ID do wykluczenia (przekazana jako side input).
        - lower_bound: dolna granica długości imienia.
        - upper_bound: górna granica długości imienia.
        """

        # Rozdzielamy wiersz wejściowy na poszczególne wartości, zakładając format CSV
        id = element.split(',')[0]  # Pobieramy pierwszą kolumnę (ID pracownika)
        name = element.split(',')[1]  # Pobieramy drugą kolumnę (Imię pracownika)

        # Usuwamy ewentualne błędy kodowania znaków
        # id = id.decode('utf-8', 'ignore').encode("utf-8")
        id = id.encode("utf-8")

        # Tworzymy listę elementów, które później zwrócimy, jeśli spełnią warunki
        element_list = element.split(',')

        # Sprawdzamy, czy długość imienia mieści się w podanym przedziale oraz
        # czy ID nie znajduje się na liście wykluczonych pracowników
        if (lower_bound <= len(name) <= upper_bound) and id not in side_list:
            return [element_list]  # Zwracamy tylko poprawne wiersze

# Definiujemy przetwarzanie danych w potoku Beam
small_names = (
    p
    | "Read from text file" >> beam.io.ReadFromText('dept_data.txt')  # Wczytujemy dane wejściowe z pliku tekstowego
    | "ParDo with side inputs" >> beam.ParDo(FilterUsingLength(), side_list, 3, 10)  # Filtrowanie za pomocą ParDo i side inputs
    | beam.Filter(lambda record: record[3] == 'Accounts')  # Zachowujemy tylko pracowników z działu 'Accounts'
    | beam.Map(lambda record: (record[0] + " " + record[1], 1))  # Mapujemy ID + imię jako klucz, wartość = 1
    | beam.CombinePerKey(sum)  # Sumujemy wartości dla każdego unikalnego klucza (ID + imię)
    | 'Write results' >> beam.io.WriteToText('output_new_final')  # Zapisujemy wyniki do pliku wyjściowego
)

# Uruchamiamy potok
p.run()

In [4]:
!{('head -n 20 /content/output_new_final-00000-of-00001')}

('149633CM Marco', 31)
('212539MU Rebekah', 31)
('231555ZZ Itoe', 31)
('503996WI Edouard', 31)
('704275DC Kyle', 31)
('957149WC Kyle', 31)
('241316NX Kumiko', 31)
('796656IE Gaston', 31)
('718737IX Ayumi', 30)


##Dodatkowe wyjścia (additional outputs)

Do tej pory widzieliśmy, że transformacja zwraca tylko jedno główne wyjście `PCollection`. Jednak możliwe jest wygenerowanie wielu dodatkowych wyjściowych `PCollection` obok głównego. Jeśli zdecydujemy się na wiele wyjść, nasza transformacja zwróci wszystkie kolekcje `PCollection` razem w pakiecie. Aby emitować elementy do wielu wyjściowych `PCollection`, używamy funkcji `with_outputs()` w transformacji ParDo i określamy różne tagi dla wyjść.

Przykład – filtrowanie imion według różnych warunków. Rozważmy potok, który odczytuje plik departamentów.

Chcemy podzielić dane na trzy grupy:
1.	Krótkie imiona – imiona o długości do 4 znaków.
2.	Długie imiona – imiona o długości powyżej 4 znaków.
3.	Imiona zaczynające się na “A”.

Ten przypadek można rozwiązać na dwa sposoby:
1.	Poprzez rozgałęzienie potoku (`branched pipelines`) – wymagałoby to więcej kodu.
2.	Poprzez dodatkowe wyjścia (`additional outputs`), co jest bardziej eleganckie.

In [19]:
import apache_beam as beam  # Importujemy Apache Beam

# Definiujemy klasę ProcessWords dziedziczącą po beam.DoFn, aby przetwarzać dane w potoku.
class ProcessWords(beam.DoFn):
    def process(self, element, cutoff_length, marker):
        """
        Funkcja process() jest wywoływana dla każdego elementu w PCollection.
        Przetwarza wiersz wejściowy i przydziela go do odpowiednich kategorii.

        :param element: pojedynczy wiersz danych wejściowych (np. "101,Anna,HR")
        :param cutoff_length: maksymalna długość krótkiego imienia (np. 4)
        :param marker: litera, na którą ma zaczynać się imię (np. "A")
        :return: odpowiednio otagowane wartości PCollection
        """
        name = element.split(',')[1]  # Pobieramy drugą kolumnę, czyli imię

        if len(name) <= cutoff_length:
            # Jeśli długość imienia jest mniejsza lub równa cutoff_length, zwracamy je jako Short_Names
            return [beam.pvalue.TaggedOutput('Short_Names', name)]

        else:
            # W przeciwnym razie zwracamy je jako Long_Names
            return [beam.pvalue.TaggedOutput('Long_Names', name)]

        if name.startswith(marker):
            # Jeśli imię zaczyna się na określony znak (np. 'M'), zwracamy je jako główne wyjście
            return name

# Tworzymy potok Apache Beam
p = beam.Pipeline()

# Przetwarzamy dane wejściowe
results = (
    p
    | "Read File" >> beam.io.ReadFromText('/content/dept_data.txt')  # Odczytujemy dane wejściowe z pliku dept_data.txt
    | "Process Names" >> beam.ParDo(ProcessWords(), cutoff_length=4, marker='M')
        .with_outputs('Short_Names', 'Long_Names', main='Names_M')  # Definiujemy tagi dla dodatkowych wyjść
)

# Przypisujemy poszczególne kolekcje do zmiennych na podstawie tagów
short_collection = results.Short_Names  # PCollection z krótkimi imionami
long_collection = results.Long_Names  # PCollection z długimi imionami
startM_collection = results.Names_M  # PCollection z imionami zaczynającymi się na "M"

# Zapisujemy wyniki do plików
short_collection | 'Write Short Names' >> beam.io.WriteToText('short')
long_collection | 'Write Long Names' >> beam.io.WriteToText('long')
startM_collection | 'Write Names Starting With M' >> beam.io.WriteToText('start_m')

# Uruchamiamy potok
p.run()

In [9]:
ls

dept_data.txt  long-00000-of-00001  sample_data/  short-00000-of-00001  start_m-00000-of-00001


In [20]:
!{('head -n 5 long-00000-of-00001')}

Marco
Rebekah
Edouard
Kumiko
Gaston


In [21]:
!{('head -n 5 short-00000-of-00001')}

Itoe
Kyle
Kyle
Olga
Kirk


In [22]:
!{('head -n 5 start_m-00000-of-00001')}